In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re


from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# finetune

In [ ]:
sw = stopwords.words('russian')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^а-яА-Я?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

## sent

In [ ]:
train = pd.read_csv('../data/raw/train.csv')

In [ ]:
new_df = train[['sentence', 'sentiment']]
new_df['sentence'] = new_df['sentence'].apply(lambda x: clean_text(x))

In [ ]:
dict_lavel = {'+':2, '?':1, '−':0}
new_df['sentiment'] = new_df['sentiment'].apply(lambda x: dict_lavel[x])
new_df

,sentence,sentiment
0,получал качественные услуги,2
1,"вижу, хотя поставить, сервис",0
2,любимый банк мкб обманул,0
3,отвратительное отношение клиентам,0
4,"любое время дня ночи помогут, ответят, решат",2
...,...,...
19356,коем случае открывайте счет недостойном довери...,0
19357,ти откровенно забили качество развивают свои м...,0
19358,"считаю, это прорыв лидерство финансовых услуг ...",2
19359,"писал мужчина очень доходчиво, финансовым язык...",2


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 24
VALID_BATCH_SIZE = 48
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('ai-forever/ruRoberta-large', truncation=True, do_lower_case=True)

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.sentence
        self.targets = self.data.sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=42)

test_data=new_df.drop(train_data.index)
test_data_index = test_data.index.tolist()
test_data=test_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (19361, 2)
TRAIN Dataset: (15489, 2)
TEST Dataset: (3872, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class MTnluClass(torch.nn.Module):
    def __init__(self):
        super(MTnluClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("ai-forever/ruRoberta-large")
        self.pre_classifier = torch.nn.Linear(1024, 1024)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(1024, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
    def get_embed(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        return pooler
    
model = MTnluClass()
model.to(device)
print('ok')

Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be 

ok


In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
LEARNING_RATE = 1e-05
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calc_roc_auc(preds, targets):
    # proba_dict = {0:np.array([1.,0.,0.]),
    #               1:np.array([0.,1.,0.]),
    #               2:np.array([0.,0.,1.])}
    # targets = np.array(list(map(lambda x: proba_dict[x], targets.cpu().numpy())))
    # preds = np.array(list(map(lambda x: proba_dict[x], preds.cpu().numpy())))
    # print(preds)
    # print(targets)
    y_preds = label_binarize(preds.cpu().numpy(), classes=[0,1,2])
    return roc_auc_score(y_preds, targets.cpu().numpy(), multi_class='ovr')

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model
epochs = 5
for epoch in range(0, epochs):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        y_preds = label_binarize(big_idx.cpu().numpy(), classes=[0,1,2])
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss per 5000 steps: {loss_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        
    y_preds_test = []
    targets_test = []
    for _,data in tqdm(enumerate(testing_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            big_val, big_idx = torch.max(outputs.data, dim=1)
            y_preds = label_binarize(big_idx.cpu().numpy(), classes=[0,1,2])
            y_preds_test += y_preds.tolist()
            targets_test += targets.cpu().tolist()
            
    epoch_loss = tr_loss/nb_tr_steps     
    print(f"Training Loss Epoch: {epoch_loss}") 
    print(f"valid Roc Auc Epoch: {roc_auc_score(np.array(targets_test), np.array(y_preds_test), multi_class='ovr')}")
    #print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    
    # epoch_accu = (n_correct*100)/nb_tr_examples


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training Loss per 5000 steps: 0.1246979609131813


646it [07:52,  1.37it/s]
81it [00:44,  1.84it/s]


Training Loss Epoch: 0.28332379932051105
valid Roc Auc Epoch: 0.8806997916189184


0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.2563576400279999


116it [01:24,  1.37it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), '/app/hsehack_2023/models/rorubert88_n.pth')

## cat

In [ ]:
train = pd.read_csv('../data/raw/train.csv')

In [ ]:
new_df = train[['sentence', 'sentiment']]
new_df['sentence'] = new_df['sentence'].apply(lambda x: clean_text(x))

In [ ]:
dict_lavel = {'+':2, '?':1, '−':0}
new_df['sentiment'] = new_df['sentiment'].apply(lambda x: dict_lavel[x])
new_df

,sentence,sentiment
0,получал качественные услуги,2
1,"вижу, хотя поставить, сервис",0
2,любимый банк мкб обманул,0
3,отвратительное отношение клиентам,0
4,"любое время дня ночи помогут, ответят, решат",2
...,...,...
19356,коем случае открывайте счет недостойном довери...,0
19357,ти откровенно забили качество развивают свои м...,0
19358,"считаю, это прорыв лидерство финансовых услуг ...",2
19359,"писал мужчина очень доходчиво, финансовым язык...",2


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 24
VALID_BATCH_SIZE = 48
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('ai-forever/ruRoberta-large', truncation=True, do_lower_case=True)

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.sentence
        self.targets = self.data.sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=42)

test_data=new_df.drop(train_data.index)
test_data_index = test_data.index.tolist()
test_data=test_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (19361, 2)
TRAIN Dataset: (15489, 2)
TEST Dataset: (3872, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class MTnluClass(torch.nn.Module):
    def __init__(self):
        super(MTnluClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("ai-forever/ruRoberta-large")
        self.pre_classifier = torch.nn.Linear(1024, 1024)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(1024, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
    def get_embed(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        return pooler
    
model = MTnluClass()
model.to(device)
print('ok')

Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be 

ok


In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
LEARNING_RATE = 1e-05
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calc_roc_auc(preds, targets):
    # proba_dict = {0:np.array([1.,0.,0.]),
    #               1:np.array([0.,1.,0.]),
    #               2:np.array([0.,0.,1.])}
    # targets = np.array(list(map(lambda x: proba_dict[x], targets.cpu().numpy())))
    # preds = np.array(list(map(lambda x: proba_dict[x], preds.cpu().numpy())))
    # print(preds)
    # print(targets)
    y_preds = label_binarize(preds.cpu().numpy(), classes=[0,1,2])
    return roc_auc_score(y_preds, targets.cpu().numpy(), multi_class='ovr')

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model
epochs = 5
for epoch in range(0, epochs):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        y_preds = label_binarize(big_idx.cpu().numpy(), classes=[0,1,2])
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training Loss per 5000 steps: {loss_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        
    y_preds_test = []
    targets_test = []
    for _,data in tqdm(enumerate(testing_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            big_val, big_idx = torch.max(outputs.data, dim=1)
            y_preds = label_binarize(big_idx.cpu().numpy(), classes=[0,1,2])
            y_preds_test += y_preds.tolist()
            targets_test += targets.cpu().tolist()
            
    epoch_loss = tr_loss/nb_tr_steps     
    print(f"Training Loss Epoch: {epoch_loss}") 
    print(f"valid Roc Auc Epoch: {roc_auc_score(np.array(targets_test), np.array(y_preds_test), multi_class='ovr')}")
    #print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    
    # epoch_accu = (n_correct*100)/nb_tr_examples


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training Loss per 5000 steps: 0.1246979609131813


646it [07:52,  1.37it/s]
81it [00:44,  1.84it/s]


Training Loss Epoch: 0.28332379932051105
valid Roc Auc Epoch: 0.8806997916189184


0it [00:00, ?it/s]

Training Loss per 5000 steps: 0.2563576400279999


116it [01:24,  1.37it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), '/app/hsehack_2023/models/rorubert88_n.pth')

# get embedds

## sent

## cat

# train Catboost

## sent

## cat

# predict

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hdyst\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.sentence
        self.targets = self.data.sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
class MTnluClass(torch.nn.Module):
    def __init__(self):
        super(MTnluClass, self).__init__()
        # self.l1 = RobertaModel.from_pretrained("ai-forever/ruRoberta-large")
        self.pre_classifier = torch.nn.Linear(1024, 1024)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(1024, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
    def get_embed(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        return pooler
    
model = MTnluClass()
model.load_state_dict(torch.load('/app/hsehack_2023/models/rorubert87.pth', map_location='cpu'))
model.to(device)
print('ok')

In [ ]:
sw = stopwords.words('russian')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^а-яА-Я?.!,¿]+", " ", text) # replacing everything with space except (а-я, А-Я, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

In [ ]:
df = pd.read_csv('../data/raw/train.csv')
df = df.iloc[:10, :]
# train

In [ ]:
new_df = df[['sentence', 'sentiment']]
new_df['sentence'] = new_df['sentence'].apply(lambda x: clean_text(x))

In [ ]:
dict_label = {'+':2, '?':1, '−':0}
new_df['sentiment'] = new_df['sentiment'].apply(lambda x: dict_label[x])
new_df

,sentence,sentiment
0,получал качественные услуги,2
1,"вижу, хотя поставить, сервис",0
2,любимый банк мкб обманул,0
3,отвратительное отношение клиентам,0
4,"любое время дня ночи помогут, ответят, решат",2
5,"время согласовывалось, вс делалось быстро",2
6,абсолютное бездействие нежелание банка работат...,0
7,первая операция внесение руб успешно выполнена,1
8,почему должен звонить платить деньги музыку пл...,0
9,получив карту кредит кармане две бесплатных ка...,2


In [ ]:
MAX_LEN = 256
TEST_BATCH_SIZE = 48
tokenizer = RobertaTokenizer.from_pretrained('ai-forever/ruRoberta-large', truncation=True, do_lower_case=True)

In [ ]:
test_data=new_df
test_data_index = test_data.index.tolist()
test_data=test_data.reset_index(drop=True)

testing_set = SentimentData(test_data, tokenizer, MAX_LEN)
# testing_set[0]

In [ ]:
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
testing_loader = DataLoader(testing_set, **test_params)